Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

In [8]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [37]:
data=pd.read_csv('checkins.dat',sep='|', skiprows=[1], skipinitialspace=True)
#skipinitialspace пропуска пустые значения после разделителя(иначе пустая строка, а так NAn)

In [39]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
1,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
2,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
3,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
4,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


In [40]:
data=data.dropna() # удалили NAn

In [41]:
data.shape

(396634, 6)

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift.  Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [42]:
from sklearn.cluster import MeanShift

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [43]:
data=data.iloc[0:100000] # ок. оставим ток 100 000 строк

In [44]:
data.shape

(100000, 6)

In [55]:
data.columns=data.columns.str.strip() # какая то херня с названием строк. удалил символы лишние

In [60]:
cluster = MeanShift(bandwidth=0.1, n_jobs=-1)

In [61]:
cluster.fit(data[['latitude','longitude']]) # обучаем

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=-1, seeds=None)

In [63]:
cluster.labels_ # номера кластеров

array([ 5,  7, 30, ..., 25, 19,  4], dtype=int64)

In [64]:
cluster.cluster_centers_

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [65]:
cluster.cluster_centers_.shape # 3231 кластера. чот дохера

(3231, 2)

In [66]:
cluster.labels_.shape

(100000,)

In [67]:
cluster

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=-1, seeds=None)

In [68]:
from itertools import groupby

In [74]:
new=[i for i,g in groupby(sorted(cluster.labels_)) if len(list(g)) > 15] 
# немного sql логики.сгруппируе отсрортированные номера кластеров. получим номер классетра и число обектов с таким номером 
# далее возмьем такие номера кластеров (меток) , обхектов в которых более 15

In [96]:
len(new) # 592 искомых кластера

592

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)



In [87]:
offices=[[33.751277, -118.188740],[25.867736, -80.324116],[51.503016, -0.075479],[52.378894, 4.885084], [39.366487, 117.036146],[-33.868457, 151.205134]]

In [88]:
offices

[[33.751277, -118.18874],
 [25.867736, -80.324116],
 [51.503016, -0.075479],
 [52.378894, 4.885084],
 [39.366487, 117.036146],
 [-33.868457, 151.205134]]

In [90]:
centers = np.empty((len(new), 2))
for i, j in enumerate(new):
    centers[i]=cluster.cluster_centers_[j]
# выписал центры 592 - х класстеров по номерах из new

In [95]:
centers

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [  41.61853175,  -88.44556818],
       [  39.2494686 ,  -77.1821271 ],
       [  38.65877915,  -76.8856871 ]])

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [103]:
distances = []

for office in offices:
    for center in centers:
        distances.append((center, np.sqrt(np.sum((center - office)**2))))

In [114]:
distances[0:5]

[(array([-33.86063043, 151.20477593]), 0.007834758163107856),
 (array([52.37296399,  4.89231722]), 0.009353316185992226),
 (array([ 25.84567226, -80.3188906 ]), 0.022674066158385495),
 (array([51.50299126, -0.12553729]), 0.05005829482278787),
 (array([  33.80987796, -118.14892381]), 0.07084773242719973)]

In [106]:
def sortSecond(val): 
    return val[1]

In [107]:
distances.sort(key=sortSecond) # отсортировал по второму значению в списке

In [108]:
distances[0]

(array([-33.86063043, 151.20477593]), 0.007834758163107856)

In [113]:
with open('answer.txt', 'w') as file:
    file.write(str('-33.86063043 151.20477593'))